# CommunityLM

This is a replication of the experiments from [CommunityLM](https://arxiv.org/abs/2209.07065) (Jiang et al. 2022), which probes partisan worldviews from language models, based on the [original repo](https://github.com/hjian42/communitylm).

Before running the notebook, please install requirements and download the data.
```bash
pip install -r requirements.txt
bash download_data.sh
```

In [ ]:
import pandas as pd
import numpy as np
import os

## Preparing ANES2020 Questions

This is data from the American National Election Study (ANES)

Website: https://electionstudies.org/
Email:   anes@electionstudies.org


In [ ]:
df = pd.read_csv("data/anes_pilot_2020ets_csv.csv")

print("Number of Rows", df.shape)

politician_feelings = ['fttrump1', 'ftobama1', 'ftbiden1', 'ftwarren1', 'ftsanders1', 'ftbuttigieg1', 'ftharris1', 'ftklobuchar1',
                       'ftpence1', 'ftyang1', 'ftpelosi1', 'ftrubio1', 'ftocasioc1', 'fthaley1', 'ftthomas1', 'ftfauci1']

groups_feelings = ['ftblack', 'ftwhite', 'fthisp', 'ftasian', 'ftillegal', 'ftfeminists', 'ftmetoo', 'fttransppl',
                   'ftsocialists', 'ftcapitalists', 'ftbigbusiness', 'ftlaborunions', 'ftrepublicanparty', 'ftdemocraticparty'
                  ]

partisanship = ['pid1r']

# only look self identified partisans 2144/3080. 1: Repub; 2: Democrat
df = df[df.pid1r < 3]
df.pid1r = df.pid1r.map({1: "Republican", 2: "Democrat"})
df.shape

In [ ]:
df[groups_feelings]

In [ ]:
# 999 stands for missing values
df_politician_results = df[partisanship+politician_feelings+groups_feelings].replace(999, np.nan).groupby("pid1r").mean().T
df_politician_results['is_repub_leading'] = (df_politician_results.Republican > df_politician_results.Democrat)
print(df_politician_results.shape)
df_politician_results


In [ ]:
df_politician_results.shape

In [ ]:
df_politician_results['Prompt1'] = [
    "Donald Trump",
    "Barack Obama",
    "Joe Biden",
    "Elizabeth Warren",
    "Bernie Sanders",
    "Pete Buttigieg",
    "Kamala Harris",
    "Amy Klobuchar",
    "Mike Pence",
    "Andrew Yang",
    "Nancy Pelosi",
    "Marco Rubio",
    "Alexandria Ocasio-Cortez",
    "Nikki Haley",
    "Clarence Thomas",
    "Dr. Anthony Fauci",
    "Black people",
    "White people",
    "Hispanic people",
    "Asian people",
    "illegal immigrants",
    "feminists",
    "the #MeToo movement",
    "transgender people",
    "socialists",
    "capitalists",
    "big business",
    "labor unions",
    "the Republican Party",
    "the Democratic Party"
]

df_politician_results['Prompt2'] = [
    "Donald Trump is",
    "Barack Obama is",
    "Joe Biden is",
    "Elizabeth Warren is",
    "Bernie Sanders is",
    "Pete Buttigieg is",
    "Kamala Harris is",
    "Amy Klobuchar is",
    "Mike Pence is",
    "Andrew Yang is",
    "Nancy Pelosi is",
    "Marco Rubio is",
    "Alexandria Ocasio-Cortez is",
    "Nikki Haley is",
    "Clarence Thomas is",
    "Dr. Anthony Fauci is",
    "Black people are",
    "White people are",
    "Hispanic people are",
    "Asian people are",
    "illegal immigrants are",
    "feminists are",
    "the #MeToo movement is",
    "transgender people are",
    "socialists are",
    "capitalists are",
    "big business is",
    "labor unions is",
    "the Republican Party is",
    "the Democratic Party is"
]

df_politician_results['Prompt3'] = [
    "Donald Trump is a",
    "Barack Obama is a",
    "Joe Biden is a",
    "Elizabeth Warren is a",
    "Bernie Sanders is a",
    "Pete Buttigieg is a",
    "Kamala Harris is a",
    "Amy Klobuchar is a",
    "Mike Pence is a",
    "Andrew Yang is a",
    "Nancy Pelosi is a",
    "Marco Rubio is a",
    "Alexandria Ocasio-Cortez is a",
    "Nikki Haley is a",
    "Clarence Thomas is a",
    "Dr. Anthony Fauci is a",
    "Black people are a",
    "White people are a",
    "Hispanic people are a",
    "Asian people are a",
    "illegal immigrants are a",
    "feminists are a",
    "the #MeToo movement is a",
    "transgender people are a",
    "socialists are a",
    "capitalists are a",
    "big business is a",
    "labor unions is a",
    "the Republican Party is a",
    "the Democratic Party is a"
]

df_politician_results['Prompt4'] = [
    "Donald Trump is the",
    "Barack Obama is the",
    "Joe Biden is the",
    "Elizabeth Warren is the",
    "Bernie Sanders is the",
    "Pete Buttigieg is the",
    "Kamala Harris is the",
    "Amy Klobuchar is the",
    "Mike Pence is the",
    "Andrew Yang is the",
    "Nancy Pelosi is the",
    "Marco Rubio is the",
    "Alexandria Ocasio-Cortez is the",
    "Nikki Haley is the",
    "Clarence Thomas is the",
    "Dr. Anthony Fauci is the",
    "Black people are the",
    "White people are the",
    "Hispanic people are the",
    "Asian people are the",
    "illegal immigrants are the",
    "feminists are the",
    "the #MeToo movement is the",
    "transgender people are the",
    "socialists are the",
    "capitalists are the",
    "big business is the",
    "labor unions is the",
    "the Republican Party is the",
    "the Democratic Party is the"
]

df_politician_results['pid'] = df_politician_results.index
# make the output directory if it doesn't exist
if not os.path.exists("output"):
    os.makedirs("output")
df_politician_results.to_csv("output/anes2020_pilot_prompt_probing.csv", index=False)
df_politician_results

In [ ]:
df_politician_results['diff'] = (df_politician_results.Democrat-df_politician_results.Republican).apply(abs)
df_politician_results.sort_values(by=['diff'])

## Generate predictions

Generate predictions from the CommunityLM models.
TODO: This is not implemented in LLMents yet, but needs to be.

In [ ]:
# TODO: Because this is not implemented yet, we just download them from the original repo in download_data.sh
df_dem = pd.read_csv("output/finetuned_gpt2_2019_dem/finetuned_gpt2_group_stance_predictions.csv")
df_repub = pd.read_csv("output/finetuned_gpt2_2019_repub/finetuned_gpt2_group_stance_predictions.csv")

## Evaluate fine-tuned GPT-2 CommunityLM models

This evaluates the sentiment of the completions generated by each model according to a sentiment classification model.

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def compute_scores(df_anes, df_dem, df_repub):
    df_repub['prediction'] = (df_repub['group_sentiment'] > df_dem['group_sentiment'])

    gold_labels = df_anes.is_repub_leading.astype(int).values
    rows = []
    for run in range(1, 6):
        run = "run_{}".format(run)
        for prompt_format in range(1, 5):
            prompt_format = "Prompt{}".format(prompt_format)
            df_ = df_repub[(df_repub.run == run) & (df_repub.prompt_format == prompt_format)]
            pred_labels = df_.prediction.astype(int).values
            acc = accuracy_score(gold_labels, pred_labels) 
            p, r, f1, _ = precision_recall_fscore_support(gold_labels, pred_labels, average='weighted')
            rows.append([run, prompt_format, acc, p, r, f1])
    df_scores = pd.DataFrame(rows, columns=["run", "prompt_format", "accuracy", "precision", "recall", "f1"])
    return df_scores

### using `cardiffnlp/twitter-roberta-base-sentiment-latest` sentiment classifier

In [ ]:
df = pd.read_csv("output/anes2020_pilot_prompt_probing.csv")
df_scores = compute_scores(df, df_dem, df_repub)
df_scores

In [ ]:
# extract gold ranks
df_politician_results = df_politician_results.sort_values(by=["pid"])
gold_dem_rank = df_politician_results['Democrat'].rank().values
gold_repub_rank = df_politician_results['Republican'].rank().values
gold_repub_rank

from scipy import stats
def extract_ranking(df_):
    df_ = df_.sort_values(by=['question'])
    return df_[df_.prompt_format == "Prompt4"].groupby(['question']).group_sentiment.mean().rank().values

dem_rank = extract_ranking(df_dem)
repub_rank = extract_ranking(df_repub)

gold_dem_rank

In [ ]:
## plot the rankings

def extract_ranking_for_politicians(df_):
    df_ = df_[df_.question.isin(politician_feelings)]
    df_ = df_.sort_values(by=['question', 'run'])
    return df_[df_.prompt_format == "Prompt4"]

df_politician_results = df_politician_results[df_politician_results.pid.isin(politician_feelings)].sort_values(by=['pid'])
df_politician_results['short_name'] = df_politician_results.Prompt1.apply(lambda x: x.split(" ")[-1])

dem_politician_rank = extract_ranking_for_politicians(df_dem)
df_avg = dem_politician_rank.groupby("question").group_sentiment.mean().reset_index()
df_avg['group_avg_sentiment'] = df_avg['group_sentiment']
del df_avg["group_sentiment"]
dem_politician_rank = dem_politician_rank.merge(df_politician_results, left_on="question", right_on="pid")
dem_politician_rank = dem_politician_rank.merge(df_avg, on="question")


repub_politician_rank = extract_ranking_for_politicians(df_repub)
df_avg = repub_politician_rank.groupby("question").group_sentiment.mean().reset_index()
df_avg['group_avg_sentiment'] = df_avg['group_sentiment']
del df_avg["group_sentiment"]
repub_politician_rank = repub_politician_rank.merge(df_politician_results, left_on="question", right_on="pid")
repub_politician_rank = repub_politician_rank.merge(df_avg, on="question")


dem_politician_rank

In [ ]:
# df_politician_results.to_csv("rank_plots.csv")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(5,5)})

palette = sns.color_palette("Blues",n_colors=20)
palette.reverse()

ax = sns.barplot(data=dem_politician_rank.sort_values(by="group_avg_sentiment", ascending=False), x="group_sentiment", y="short_name", palette=palette, estimator=np.mean, ci=90)

ax.set_xlabel(None, fontsize=15)
ax.set_ylabel(None)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
plt.savefig('rankings/finetuned_gpt2_pred_dem_rank.png', bbox_inches = "tight")

In [ ]:
sns.set(rc={'figure.figsize':(5,5)})

palette = sns.color_palette("Blues",n_colors=20)
palette.reverse()

ax = sns.barplot(data=dem_politician_rank.sort_values(by="Democrat", ascending=False), x="Democrat", y="short_name", palette=palette)

ax.set_xlabel(None, fontsize=15)
ax.set_ylabel(None)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
plt.savefig('rankings/gold_dem_rank.png', bbox_inches = "tight")

In [ ]:
palette = sns.color_palette("Reds", n_colors=20)
palette.reverse()

ax = sns.barplot(data=repub_politician_rank.sort_values(by="group_avg_sentiment", ascending=False), x="group_sentiment", y="short_name", palette=palette)

ax.set_xlabel(None, fontsize=15)
ax.set_ylabel(None)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
plt.savefig('rankings/finetuned_gpt2_pred_repub_rank.png', bbox_inches = "tight")

In [ ]:
palette = sns.color_palette("Reds", n_colors=20)
palette.reverse()

ax = sns.barplot(data=repub_politician_rank.sort_values(by="Republican", ascending=False), x="Republican", y="short_name", palette=palette)

ax.set_xlabel(None, fontsize=15)
ax.set_ylabel(None)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
plt.savefig('rankings/gold_repub_rank.png', bbox_inches = "tight")